<a href="https://colab.research.google.com/github/BoonthichaSaejia/ThaiSum/blob/master/ThaiSum_0822_TestThaiSum_with_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 00 Setup and Download data

In [ ]:
#clone git repository of ThaiSum
%%capture
!git clone https://github.com/nakhunchumpolsathien/ThaiSum.git

In [ ]:
#install some library that don't have in colab
!pip install -q transformers
!pip install -q pythainlp
!pip install -q folium==0.2.1
!pip install -q pytorch_transformers
!pip install -q tensorboardX
!pip install -q pyrouge
!pyrouge_set_rouge_path "/content/ThaiSum/BertSum/ROUGE-1.5.5"
!apt update
!apt install -q libxml-parser-perl
%cd "/content/ThaiSum/BertSum/ROUGE-1.5.5/data"
!perl WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db

     |████████████████████████████████| 2.6 MB 7.0 MB/s 
     |████████████████████████████████| 636 kB 48.0 MB/s 
     |████████████████████████████████| 3.3 MB 54.6 MB/s 
     |████████████████████████████████| 895 kB 78.8 MB/s 
     |████████████████████████████████| 11.0 MB 7.1 MB/s 
     |████████████████████████████████| 743 kB 58.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires typing-extensions~=3.7.4, but you have typing-extensions 3.10.0.0 which is incompatible.
     |████████████████████████████████| 69 kB 5.4 MB/s 
     |████████████████████████████████| 176 kB 7.4 MB/s 
     |████████████████████████████████| 131 kB 58.2 MB/s 
     |████████████████████████████████| 1.2 MB 51.2 MB/s 
     |████████████████████████████████| 7.8 MB 37.6 MB/s 
     |████████████████████████████████| 79 kB 9.4 MB/s 
     |██████████████

In [ ]:
#bertEx model to model_checkpoint
%cd /content/ThaiSum/BertSum/model_checkpoint
!gdown --id 1P4amtaHcvNc_UOscZZMMyWutRZuohc1P

/content/ThaiSum/BertSum/model_checkpoint
Downloading...
From: https://drive.google.com/uc?id=1P4amtaHcvNc_UOscZZMMyWutRZuohc1P
To: /content/ThaiSum/BertSum/model_checkpoint/BertsumExt_thaisum.pt
2.28GB [00:29, 76.2MB/s]


In [ ]:
#go back until at /content
%cd ..

/content


In [ ]:
# download train data
# !gdown --id 1MOp7gOE2W7vpRRWVtTWkbhy8nreQleZS
#download test data
!gdown --id 1OJN1U2y5vbE2MqDCYBhe-wtvWYsIcjc4
#download validate data
# !gdown --id 1_t6xtPOys3ZKCTEB7rB0rqTVVvXthKyo

Downloading...
From: https://drive.google.com/uc?id=1OJN1U2y5vbE2MqDCYBhe-wtvWYsIcjc4
To: /content/test_set.csv
119MB [00:03, 34.9MB/s]


# **01 Get Testset**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
testset = pd.read_csv("test_set.csv").head(2000)
testset = testset[['body','summary']]
testset

,body,summary
0,"พอไม่ได้ดังใจก็ร่างใหม่ แสลงชื่อสมชัยเดินหนี,ม...",มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ...
1,กีเก ซานเชซ ฟลอเรส กุนซือเลือดกระทิงของทีมวัต...,กีเก ซานเชซ ฟลอเรส กุนซือเลือดกระทิงของทีมวัต...
2,"ตาย30-เจ็บ400 เส้นทางตัดขาด เข้าช่วยลำบาก,เนปา...",เนปาลระทึก หลังเกิดพายุฝนฟ้าคะนองถล่มทางใต้ สร...
3,สมเด็จพระเทพรัตนราชสุดาฯ สยามบรมราชกุมารี ทรงเ...,เป็นศูนย์รวมของความคิดสร้างสรรค์และคอมมูนิตี้ค...
4,ทุกคนอาจจะต้องการเพียงแค่ใช้ชีวิตในช่วงบั้นปล...,สหประชาชาติประเมินว่า สังคมเมียนมากำลังก้าวเข้...
...,...,...
1995,5ปีก่อนโดนตรวจเจอ225ไร่ รอคดีเงียบลุยล้ำไม่แยแ...,กองปราบฯ ร่วมเจ้าหน้าที่อุทยานฯเขาใหญ่ ฝ่ายปกค...
1996,นิตยสารฉลาดซื้อ และศูนย์พิทักษ์สิทธิผู้บริโภค ...,ผลสำรวจการแสดงฉลากของอาหารกึ่งสำเร็จรูปชนิดจั...
1997,ชาวบ้านร้อง ดีเอสไอ รับคดี โครงการออมทรัพย์ที่...,ชาวบ้านร้องดีเอสไอ รับคดี โครงการออมทรัพย์ที่อ...
1998,"หวั่นผู้ค้ามั่วนิ่มขึ้นราคาอ้างน้ำมันพุ่ง,สมคิ...",สมคิด สั่ง พาณิชย์ คุมเข้มราคาสินค้า ไม่ให้ฉวย...


In [ ]:
#see raw_text
testset.iloc[0,0]

'พอไม่ได้ดังใจก็ร่างใหม่ แสลงชื่อสมชัยเดินหนี,มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง 34 สนช.ยื่นตีความต่อวาระให้ผู้ตรวจฯ ส่อขัด รธน. พท.ดาหน้าฉะ ซือแป๋ ใครแน่ดูถูกประชาชน สมคิด ซัดพูดย้อนแย้งไหลไปไหลมา ลั่น ไม่เคยกลัวในสนามเลือกตั้ง สมชัย กระแทกซ้ำเปิดช่องโกงสนั่นกันทั้งประเทศ ทนายปู มึนรถตู้รับจ้างถูกเล่นงาน ภูมิธรรม ดับกระแส พท.วางตัวผู้นำ ยันพรรคเหนียวแน่นไม่มีปีนเกลียว วิม แจงไม่ได้ล็อกเป้าแค่คน ชินวัตร บิ๊กตู่ รอรายงาน โกตี๋ ตายที่ลาว บิ๊กป้อม โบ้ยศพใครไม่รู้ ผบ.ทบ.ชี้ภาพตกแต่งกันได้ ครม.ตั้ง เลิศวิโรจน์ ขึ้นปลัดเกษตรฯ สุทธิพงษ์ อ.ส่งเสริมปกครองท้องถิ่น ประยุทธ์ รับ รมว.ต่างประเทศสหรัฐฯเยือนไทย ปัดไม่ได้โดนรังเกียจจากนานาชาติ ดอน ยังมั่นใจไปเยือนในปีนี้,แม้เสียงคัดค้านจากฝ่ายการเมืองต่อรูปแบบการเลือกตั้ง ส.ส. สูตรพิสดารของกรรมการร่างรัฐธรรมนูญ (กรธ.) ยังคงดังกระหึ่ม แต่นายมีชัย ฤชุพันธุ์ ประธาน กรธ. ยังคงยืนยันเดินหน้าต่อไป แม้การเลือกตั้งตามรูปแบบใหม่

In [ ]:
#see tgt
testset.iloc[0,1]

'มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง'

**NOTICE : summary is from the fitst 1-2 sentence**

## 1.1 Preprocessing

In [ ]:
import re
import pandas as pd
import pythainlp
import operator
import math
from tqdm import tqdm
from timeit import default_timer as timer
import time

from pythainlp.tokenize import sent_tokenize
from pythainlp.tokenize import subword_tokenize
from pythainlp.tokenize import word_tokenize
import transformers

import json
from google.colab import files

In [ ]:
def list_to_string(list):
    string = ''.join(list)
    string = ' '.join(string.split())
    return string


def middle_cut(sentences):
        new_text = ""
        for sentence in sentences:
            sentence_size = len(pythainlp.word_tokenize(sentence, keep_whitespace=False))

            for k in range(0, len(sentence)):
                if k == 0 or k + 1 >= len(sentence):
                    continue
                if sentence[k].isdigit() and sentence[k - 1] == " ":
                    sentence = sentence[:k - 1] + sentence[k:]
                if k + 2 <= len(sentence):
                    if sentence[k].isdigit() and sentence[k + 1] == " ":
                        sentence = sentence[:k + 1] + sentence[k + 2:]

            fixed_text_lenth = 20

            if sentence_size > fixed_text_lenth:
                partition = math.floor(sentence_size / fixed_text_lenth)
                tokens = pythainlp.word_tokenize(sentence, keep_whitespace=True)
                for i in range(0, partition):
                    middle_space = (sentence_size / (partition+1)*(i+1))
                    white_space_index = []
                    white_space_diff = {}

                    for j in range(len(tokens)):
                        if tokens[j] == ' ':
                            white_space_index.append(j)

                    for white_space in white_space_index:
                        white_space_diff.update({white_space: abs(white_space - middle_space)})

                    if len(white_space_diff) > 0:
                        min_diff = min(white_space_diff.items(), key=operator.itemgetter(1))
                        tokens.pop(min_diff[0])
                        tokens.insert(min_diff[0], "<stop>")
                new_text = new_text + list_to_string(tokens) + "<stop>"
            else:
                new_text = new_text + sentence + "<stop>"

        sentences = new_text.split("<stop>")
        sentences = [s.strip() for s in sentences]
        if '' in sentences: sentences.remove('')
        if 'nan' in sentences: sentences.remove('nan')

        sentences = list(filter(None, sentences))
        return sentences

In [ ]:
class ThaiSentenceSegmentor:

    def split_into_sentences(self, text, isMiddleCut=False):
        # Declare Variables
        th_alphabets = "([ก-๙])"
        th_conjunction = "(เพราะ|ทำให้|โดย|เนื่องจาก|เพราะ|นอกจากนี้|แต่|กรณีที่|หลังจากนี้|ต่อมา|ภายหลัง|นับตั้งแต่|หลังจาก|ซึ่งเหตุการณ์|ผู้สื่อข่าวรายงานอีก|ส่วนที่|ส่วนสาเหตุ|ฉะนั้น|เพราะฉะนั้น|เพื่อ|เนื่องจาก|จากการสอบสวนทราบว่า|จากกรณี|จากนี้|อย่างไรก็ดี)"
        th_cite = "(กล่าวว่า|เปิดเผยว่า|รายงานว่า|ให้การว่า|เผยว่า|บนทวิตเตอร์ว่า|แจ้งว่า|พลเมืองดีว่า|อ้างว่า)"
        th_ka_krub = "(ครับ|ค่ะ)"
        th_stop_after = "(หรือไม่|โดยเร็ว|แล้ว|อีกด้วย)"
        th_stop_before = "(ล่าสุด|เบื้องต้น|ซึ่ง|ทั้งนี้|แม้ว่า|เมื่อ|แถมยัง|ตอนนั้น|จนเป็นเหตุให้|จากนั้น|อย่างไรก็ตาม|และก็|อย่างใดก็ตาม|เวลานี้|เช่น|กระทั่ง)"
        degit = "([0-9])"
        th_title = "(นาย|นาง|นางสาว|เด็กชาย|เด็กหญิง|น.ส.|ด.ช.|ด.ญ.|อ.|รศ.|ผศ.|ศจ.|ดร.|ผอ.|พล.ต.อ.|พล.ต.ท.|พล.ต.ต.|พล.ต.จ.|พ.ต.อ.|พ.ต.ท.|พ.ต.ต.|ร.ต.อ.|ร.ต.ท.|ร.ต.ต.|พล.อ.|พล.ท.|พล.ต.|พ.อ.|พ.ท.|พ.ต.|ร.อ.|ผบ.ทบ.|ผบ.ทร.|ผบ.ตร.|ผบ.ทสส|ผบ.ทอ.)"


        text = f" {text} "
        text = text.replace("\n", " ")
        text = text.replace("", "")
        text = text.replace("โดยเร็ว", "<rth_Doeirew>")
        text = text.replace("เพื่อน", "<rth_friend>")
        text = text.replace("โดยสาร", "<rth_passenger>")
        text = text.replace("แล้วแต่", "<rth_leawtea>")
        text = text.replace("หรือเปล่า", "<rth_repraw>")
        text = text.replace("หรือไม่", "<rth_remai>")
        text = text.replace("จึงรุ่งเรืองกิจ", "<rth_tanatorn_lastname>")
        text = text.replace("ตั้งแต่", "<rth_tangtea>")
        text = text.replace("แต่ละ", "<rth_teala>")
        text = text.replace("ชีวิตแล้ว", "<rth_chiwitleaw>")
        text = text.replace("โดยประ", "<rth_doipra>")
        text = text.replace("แต่หลังจากนั้น", "<rth_tealangjaknan>")
        text = text.replace("พรรคเพื่อ", "<for_party>")
        text = text.replace("แต่เนื่อง", "<rth_teaneung>")
        text = text.replace("เพื่อทำให้", "เพื่อ<rth_tamhai>")
        text = text.replace("ทำเพื่อ", "ทำ<rth_for>")
        text = text.replace("จึงทำให้", "จึง<tamhai>")
        text = text.replace("มาโดยตลอด", "<madoitalod>")
        text = text.replace("แต่อย่างใด", "<teayangdaikptam>")
        text = text.replace("แต่หลังจาก", "แต่<langjak>")
        text = text.replace("คงทำให้", "<rth_kongtamhai>")
        text = text.replace("แต่ทั้งนี้", "แต่<tangni>")
        text = text.replace("มีแต่", "มี<tea>")
        text = text.replace("เหตุที่ทำให้", "<hedteetamhai>")
        text = text.replace("โดยหลังจาก", "โดย<langjak>")
        text = text.replace("ซึ่งหลังจาก", "ซึ่ง<langjak>")
        text = text.replace("ตั้งโดย", "<rth_tangdoi>")
        text = text.replace("โดยตรง", "<rth_doitong>")
        text = text.replace("นั้นหรือ", "<rth_nanhlor>")
        text = text.replace("ซึ่งต้องทำให้", "ซึ่งต้อง<tamhai>")
        text = text.replace("ชื่อต่อมา", "ชื่อ<tomar>")
        text = text.replace("โดยเร่งด่วน", "<doi>เร่งด่วน")
        text = text.replace("ไม่ได้ทำให้", "ไม่ได้<tamhai>")
        text = text.replace("จะทำให้", "จะ<tamhai>")
        text = text.replace("จนทำให้", "จน<tamhai>")
        text = text.replace("เว้นแต่", "เว้น<rth_tea>")
        text = text.replace("ก็ทำให้", "ก็<tamhai>")
        text = text.replace(" ณ ตอนนั้น", " ณ <tonnan>")
        text = text.replace("บางส่วน", "บาง<rth_suan>")
        text = text.replace("หรือแม้แต่", "หรือ<rth_meatea>")
        text = text.replace("โดยทำให้", "โดย<tamhai>")
        text = text.replace("หรือเพราะ", "หรือ<rth_orbecause>")
        text = text.replace("มาแต่", "มา<rth_tea>")
        text = text.replace("แต่ไม่ทำให้", "แต่<maitamhai>")
        text = text.replace("ฉะนั้นเมื่อ", "ฉะนั้น<rth_moe>")
        text = text.replace("เพราะฉะนั้น", "เพราะ<rth_chanan>")
        text = text.replace("เพราะหลังจาก", "เพราะ<rth_langjak>")
        text = text.replace("สามารถทำให้", "สามารถ<rth_tamhai>")
        text = text.replace("อาจทำ", "อาจ<rth_tam>")
        text = text.replace("จะทำ", "จะ<rth_tam>")
        text = text.replace("และนอกจากนี้", "นอกจากนี้")
        text = text.replace("อีกทั้งเพื่อ", "อีกทั้ง<rth_for>")
        text = text.replace("ทั้งนี้เพื่อ", "ทั้งนี้<rth_for>")
        text = text.replace("เวลาต่อมา", "เวลา<rth_toma>")
        text = text.replace("อย่างไรก็ตาม", "อย่างไรก็ตาม")
        text = text.replace("อย่างไรก็ตามหลังจาก", "<stop>อย่างไรก็ตาม<rth_langjak>")
        text = text.replace("ซึ่งทำให้", "ซึ่ง<rth_tamhai>")
        text = text.replace("โดยประมาท", "<doi>ประมาท")
        text = text.replace("โดยธรรม", "<doi>ธรรม")
        text = text.replace("โดยสัจจริง", "<doi>สัจจริง")
        text = text.replace("?<stop>","?")
        text = text.replace("!<stop>","!")

        if "และ" in text:
            tokens = pythainlp.word_tokenize(text.strip(), keep_whitespace=True)
            and_position = -1
            nearest_space_position = -1
            last_position = len(tokens)
            pop_split_position = []
            split_position = []
            for i in range(len(tokens)):
                if tokens[i] == "และ":
                    and_position = i

                if and_position != -1 and i > and_position and tokens[i] == " " and nearest_space_position == -1:
                    if i - and_position != 1:
                        nearest_space_position = i

                if and_position != -1 and last_position - and_position == 3:
                    split_position.append(last_position)
                    and_position = -1
                    nearest_space_position = -1

                if nearest_space_position != -1:
                    if nearest_space_position - and_position < 5:
                        pop_split_position.append(nearest_space_position)
                    else:
                        split_position.append(and_position)
                    and_position = -1
                    nearest_space_position = -1
            for pop in pop_split_position:
                tokens.pop(pop)
                tokens.insert(pop, "<stop>")
            for split in split_position:
                tokens.insert(split, "<stop>")
            text = list_to_string(tokens)

        if "หรือ" in text:
            tokens = pythainlp.word_tokenize(text.strip(), keep_whitespace=True)
            or_position = -1
            nearest_space_position = -1
            last_position = len(tokens)
            pop_split_position = []
            split_position = []
            for i in range(len(tokens)):
                if tokens[i] == "หรือ":
                    or_position = i
                if or_position != -1 and i > or_position and tokens[i] == " " and nearest_space_position == -1:
                    if i - or_position != 1:
                        nearest_space_position = i

                if or_position != -1 and last_position - or_position == 3:
                    split_position.append(last_position)
                    or_position = -1
                    nearest_space_position = -1

                if nearest_space_position != -1:
                    if nearest_space_position - or_position < 4:
                        pop_split_position.append(nearest_space_position)
                    else:
                        split_position.append(or_position)
                    or_position = -1
                    nearest_space_position = -1
            for pop in pop_split_position:
                tokens.pop(pop)
                tokens.insert(pop, "<stop>")
            for split in split_position:
                tokens.insert(split, "<stop>")
            text = list_to_string(tokens)

        if "จึง" in text:
            tokens = pythainlp.word_tokenize(text.strip(), keep_whitespace=True)
            cung_position = -1
            nearest_space_position = -1
            pop_split_position = []
            last_position = len(tokens)
            split_position = []
            for i in range(len(tokens)):
                if tokens[i] == "จึง":
                    cung_position = i

                if cung_position != -1 and tokens[i] == " " and i > cung_position and nearest_space_position == -1:
                    if i - cung_position != 1:
                        nearest_space_position = i

                if cung_position != -1 and last_position - cung_position == 2:
                    split_position.append(last_position)
                    cung_position = -1
                    nearest_space_position = -1

                if nearest_space_position != -1:
                    if nearest_space_position - cung_position < 3:
                        pop_split_position.append(nearest_space_position)
                    else:
                        split_position.append(cung_position)
                    cung_position = -1
                    nearest_space_position = -1

            for pop in pop_split_position:
                tokens.pop(pop)
                tokens.insert(pop, "<stop>")
            for split in split_position:
                tokens.insert(split, "<stop>")

            text = list_to_string(tokens)

        text = re.sub(" " + th_stop_before, "<stop>\\1", text)
        text = re.sub(th_ka_krub, "\\1<stop>", text)
        text = re.sub(th_conjunction, "<stop>\\1", text)
        text = re.sub(th_cite, "\\1<stop>", text)
        text = re.sub(" " + degit + "[.]" + th_title, "<stop>\\1.\\2", text)
        text = re.sub(" " + degit + degit + "[.]" + th_title, "<stop>\\1\\2.\\3", text)
        text = re.sub(th_alphabets + th_stop_after + " ", "\\1\\2<stop>", text)
        if "”" in text: text = text.replace(".”", "”.")
        if "\"" in text: text = text.replace(".\"", "\".")
        if "!" in text: text = text.replace("!\"", "\"!")
        if "?" in text: text = text.replace("?\"", "\"?")
        text = text.replace("<rth_Doeirew>", "โดยเร็ว")
        text = text.replace("<rth_friend>", "เพื่อน")
        text = text.replace("<rth_passenger>", "โดยสาร")
        text = text.replace("<rth_leawtea>", "แล้วแต่")
        text = text.replace("<rth_repraw>", "หรือเปล่า")
        text = text.replace("<rth_remai>", "หรือไม่")
        text = text.replace("<rth_tanatorn_lastname>", "จึงรุ่งเรืองกิจ")
        text = text.replace("<rth_tangtea>", "ตั้งแต่")
        text = text.replace("<rth_teala>", "แต่ละ")
        text = text.replace("<rth_chiwitleaw>", "ชีวิตแล้ว")
        text = text.replace("<rth_doipra>", "โดยประ")
        text = text.replace("<rth_tealangjaknan>", "แต่หลังจากนั้น")
        text = text.replace("<for_party>", "พรรคเพื่อ")
        text = text.replace("<rth_teaneung>", "แต่เนื่อง")
        text = text.replace("เพื่อ<rth_tamhai>", "เพื่อทำให้")
        text = text.replace("ทำ<rth_for>", "ทำเพื่อ")
        text = text.replace("จึง<tamhai>", "จึงทำให้")
        text = text.replace("<madoitalod>", "มาโดยตลอด")
        text = text.replace("แต่<langjak>", "แต่หลังจาก")
        text = text.replace("แต่<tangni>", "แต่ทั้งนี้")
        text = text.replace("มี<tea>", "มีแต่")
        text = text.replace("<teayangdaikptam>", "แต่อย่างใด")
        text = text.replace("<rth_kongtamhai>", "คงทำให้")
        text = text.replace("<hedteetamhai>", "เหตุที่ทำให้")
        text = text.replace("โดย<langjak>", "โดยหลังจาก")
        text = text.replace("ซึ่ง<langjak>", "ซึ่งหลังจาก")
        text = text.replace("<rth_tangdoi>", "ตั้งโดย")
        text = text.replace("<rth_doitong>", "โดยตรง")
        text = text.replace("<rth_nanhlor>", "นั้นหรือ")
        text = text.replace("ซึ่งต้อง<tamhai>", "ซึ่งต้องทำให้")
        text = text.replace("ชื่อ<tomar>", "ชื่อต่อมา")
        text = text.replace("<doi>เร่งด่วน", "โดยเร่งด่วน")
        text = text.replace("ไม่ได้<tamhai>", "ไม่ได้ทำให้")
        text = text.replace("จะ<tamhai>", "จะทำให้")
        text = text.replace("จน<tamhai>", "จนทำให้")
        text = text.replace("เว้น<rth_tea>", "เว้นแต่")
        text = text.replace("ก็<tamhai>", "ก็ทำให้")
        text = text.replace(" ณ <tonnan>", " ณ ตอนนั้น")
        text = text.replace("บาง<rth_suan>", "บางส่วน")
        text = text.replace("หรือ<rth_meatea>", "หรือแม้แต่")
        text = text.replace("โดย<tamhai>", "โดยทำให้")
        text = text.replace("หรือ<rth_orbecause>", "หรือเพราะ")
        text = text.replace("มา<rth_tea>", "มาแต่")
        text = text.replace("แต่<maitamhai>", "แต่ไม่ทำให้")
        text = text.replace("ฉะนั้น<rth_moe>", "ฉะนั้นเมื่อ")
        text = text.replace("เพราะ<rth_chanan>", "เพราะฉะนั้น")
        text = text.replace("เพราะ<rth_langjak>", "เพราะหลังจาก")
        text = text.replace("สามารถ<rth_tamhai>", "สามารถทำให้")
        text = text.replace("อาจ<rth_tam>", "อาจทำ")
        text = text.replace("จะ<rth_tam>", "จะทำ")
        text = text.replace("อีกทั้ง<rth_for>", "อีกทั้งเพื่อ")
        text = text.replace("ทั้งนี้<rth_for>", "ทั้งนี้เพื่อ")
        text = text.replace("เวลา<rth_toma>", "เวลาต่อมา")
        text = text.replace("อย่างไรก็ตาม<rth_langjak>", "อย่างไรก็ตามหลังจาก", )
        text = text.replace("ซึ่ง<rth_tamhai>", "ซึ่งทำให้")
        text = text.replace("<doi>ประมาท", "โดยประมาท")
        text = text.replace("<doi>ธรรม", "โดยธรรม")
        text = text.replace("<doi>สัจจริง", "โดยสัจจริง")
        text = text.replace("?", "?<stop>")
        text = text.replace("!", "!<stop>")
        text = text.replace("<prd>", ".")
        sentences = text.split("<stop>")
        sentences = [s.strip() for s in sentences]
        if '' in sentences: sentences.remove('')
        if 'nan' in sentences: sentences.remove('nan')

        sentences = list(filter(None, sentences))

        if isMiddleCut:
            return middle_cut(sentences)
        else:
            return sentences

In [ ]:
def segmentor1(text):
  return sent_tokenize(text, engine="crfcut")


def segmentor2(text):
  segmentor = ThaiSentenceSegmentor()
  return segmentor.split_into_sentences(text)

def tokenizer(sent):
  return subword_tokenize(sent,engine='wangchanberta')

def preprocessing(col,segmentor_type=1):
  if segmentor_type==1:
    segmented_sents = list(map(segmentor1, col))
  elif segmentor_type==2:
    segmented_sents = list(map(segmentor2, col))

  tokenized_sents = []
  for i in segmented_sents:
    tokenized = list(map(tokenizer, i))
    tokenized_sents.append(tokenized)
  return tokenized_sents

In [ ]:
testset

,body,summary
0,"พอไม่ได้ดังใจก็ร่างใหม่ แสลงชื่อสมชัยเดินหนี,ม...",มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ...
1,กีเก ซานเชซ ฟลอเรส กุนซือเลือดกระทิงของทีมวัต...,กีเก ซานเชซ ฟลอเรส กุนซือเลือดกระทิงของทีมวัต...
2,"ตาย30-เจ็บ400 เส้นทางตัดขาด เข้าช่วยลำบาก,เนปา...",เนปาลระทึก หลังเกิดพายุฝนฟ้าคะนองถล่มทางใต้ สร...
3,สมเด็จพระเทพรัตนราชสุดาฯ สยามบรมราชกุมารี ทรงเ...,เป็นศูนย์รวมของความคิดสร้างสรรค์และคอมมูนิตี้ค...
4,ทุกคนอาจจะต้องการเพียงแค่ใช้ชีวิตในช่วงบั้นปล...,สหประชาชาติประเมินว่า สังคมเมียนมากำลังก้าวเข้...
...,...,...
1995,5ปีก่อนโดนตรวจเจอ225ไร่ รอคดีเงียบลุยล้ำไม่แยแ...,กองปราบฯ ร่วมเจ้าหน้าที่อุทยานฯเขาใหญ่ ฝ่ายปกค...
1996,นิตยสารฉลาดซื้อ และศูนย์พิทักษ์สิทธิผู้บริโภค ...,ผลสำรวจการแสดงฉลากของอาหารกึ่งสำเร็จรูปชนิดจั...
1997,ชาวบ้านร้อง ดีเอสไอ รับคดี โครงการออมทรัพย์ที่...,ชาวบ้านร้องดีเอสไอ รับคดี โครงการออมทรัพย์ที่อ...
1998,"หวั่นผู้ค้ามั่วนิ่มขึ้นราคาอ้างน้ำมันพุ่ง,สมคิ...",สมคิด สั่ง พาณิชย์ คุมเข้มราคาสินค้า ไม่ให้ฉวย...


In [ ]:
testset['body_1st_segmentor'] = preprocessing(testset.body,1)
testset['summary_1st_segmentor'] = preprocessing(testset.summary,1)
testset['body_2nd_segmentor'] = preprocessing(testset.body,2)
testset['summary_2nd_segmentor'] = preprocessing(testset.summary,2)

Downloading:   0%|          | 0.00/905k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/282 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/token_classification.py:155: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.NONE"` instead.
  f'`grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="{aggregation_strategy}"` instead.'


In [ ]:
testset

,body,summary,body_1st_segmentor,summary_1st_segmentor,body_2nd_segmentor,summary_2nd_segmentor
0,"พอไม่ได้ดังใจก็ร่างใหม่ แสลงชื่อสมชัยเดินหนี,ม...",มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ...,"[[▁พอ, ไม่ได้, ดัง, ใจ, ก็, ร่าง, ใหม่], [▁, แ...","[[▁, มีชัย, ▁ไม่, ยี่, ห, ระ, เสียง, ค้าน, ชง,...","[[▁พอ, ไม่ได้, ดัง, ใจ, ก็, ร่าง, ใหม่, ▁, แส,...","[[▁, มีชัย, ▁ไม่, ยี่, ห, ระ, เสียง, ค้าน, ชง,..."
1,กีเก ซานเชซ ฟลอเรส กุนซือเลือดกระทิงของทีมวัต...,กีเก ซานเชซ ฟลอเรส กุนซือเลือดกระทิงของทีมวัต...,"[[▁, กี, เก, ▁, ซาน, เช, ซ, ▁, ฟลอ, เรส, ▁, กุ...","[[▁, กี, เก, ▁, ซาน, เช, ซ, ▁, ฟลอ, เรส, ▁, กุ...","[[▁, กี, เก, ▁, ซาน, เช, ซ, ▁, ฟลอ, เรส, ▁, กุ...","[[▁, กี, เก, ▁, ซาน, เช, ซ, ▁, ฟลอ, เรส, ▁, กุ..."
2,"ตาย30-เจ็บ400 เส้นทางตัดขาด เข้าช่วยลำบาก,เนปา...",เนปาลระทึก หลังเกิดพายุฝนฟ้าคะนองถล่มทางใต้ สร...,"[[▁, ตาย, 30, -, เจ็บ, 400, ▁, เส้นทาง, ตัด, ข...","[[▁, เนปาล, ระทึก, ▁, หลังเกิด, พายุ, ฝนฟ้าคะน...","[[▁, ตาย, 30, -, เจ็บ, 400, ▁, เส้นทาง, ตัด, ข...","[[▁, เนปาล, ระทึก, ▁, หลังเกิด, พายุ, ฝนฟ้าคะน..."
3,สมเด็จพระเทพรัตนราชสุดาฯ สยามบรมราชกุมารี ทรงเ...,เป็นศูนย์รวมของความคิดสร้างสรรค์และคอมมูนิตี้ค...,"[[▁, สมเด็จพระ, เทพรัตน, ราชสุดาฯ, ▁, สยามบรม,...","[[▁, เป็น, ศูนย์รวม, ของ, ความคิด, สร้างสรรค์,...","[[▁, สมเด็จพระ, เทพรัตน, ราชสุดาฯ, ▁, สยามบรม,...","[[▁, เป็น, ศูนย์รวม, ของ, ความคิด, สร้างสรรค์,..."
4,ทุกคนอาจจะต้องการเพียงแค่ใช้ชีวิตในช่วงบั้นปล...,สหประชาชาติประเมินว่า สังคมเมียนมากำลังก้าวเข้...,"[[▁ทุกคน, อาจจะ, ต้องการ, เพียงแค่, ใช้ชีวิต, ...","[[▁, ส, หประชาชาติ, ประเมินว่า, ▁, สังคม, เมีย...","[[▁ทุกคน, อาจจะ, ต้องการ, เพียงแค่, ใช้ชีวิต, ...","[[▁, ส, หประชาชาติ, ประเมินว่า, ▁, สังคม, เมีย..."
...,...,...,...,...,...,...
1995,5ปีก่อนโดนตรวจเจอ225ไร่ รอคดีเงียบลุยล้ำไม่แยแ...,กองปราบฯ ร่วมเจ้าหน้าที่อุทยานฯเขาใหญ่ ฝ่ายปกค...,"[[▁, 5, ปีก่อน, โดน, ตรวจ, เจอ, 2, 25, ไร่, ▁ร...","[[▁, กองปราบฯ, ▁, ร่วม, เจ้าหน้า, ที่, อุทยานฯ...","[[▁, 5, ปีก่อน, โดน, ตรวจ, เจอ, 2, 25, ไร่, ▁ร...","[[▁, กองปราบฯ, ▁, ร่วม, เจ้าหน้า, ที่, อุทยานฯ..."
1996,นิตยสารฉลาดซื้อ และศูนย์พิทักษ์สิทธิผู้บริโภค ...,ผลสำรวจการแสดงฉลากของอาหารกึ่งสำเร็จรูปชนิดจั...,"[[▁, นิตยสาร, ฉลาด, ซื้อ, ▁และ, ศูนย์, พิทักษ์...","[[▁, ผลสํารวจ, การแสดง, ฉลาก, ของ, อาหาร, กึ่ง...","[[▁, นิตยสาร, ฉลาด, ซื้อ], [▁และ, ศูนย์, พิทัก...","[[▁, ผลสํารวจ, การแสดง, ฉลาก, ของ, อาหาร, กึ่ง..."
1997,ชาวบ้านร้อง ดีเอสไอ รับคดี โครงการออมทรัพย์ที่...,ชาวบ้านร้องดีเอสไอ รับคดี โครงการออมทรัพย์ที่อ...,"[[▁ชาวบ้าน, ร้อง, ▁, ดีเอสไอ, ▁รับ, คดี], [▁, ...","[[▁ชาวบ้าน, ร้อง, ดีเอสไอ], [▁รับ, คดี], [▁, โ...","[[▁ชาวบ้าน, ร้อง, ▁, ดีเอสไอ, ▁รับ, คดี, ▁, โค...","[[▁ชาวบ้าน, ร้อง, ดีเอสไอ, ▁รับ, คดี, ▁, โครงก..."
1998,"หวั่นผู้ค้ามั่วนิ่มขึ้นราคาอ้างน้ำมันพุ่ง,สมคิ...",สมคิด สั่ง พาณิชย์ คุมเข้มราคาสินค้า ไม่ให้ฉวย...,"[[▁, หวั่น, ผู้ค้า, มั่ว, นิ่ม, ขึ้นราคา, อ้าง...","[[▁, สมคิด, ▁, สั่ง, ▁, พาณิชย์, ▁, คุมเข้ม, ร...","[[▁, หวั่น, ผู้ค้า, มั่ว, นิ่ม, ขึ้นราคา, อ้าง...","[[▁, สมคิด, ▁, สั่ง, ▁, พาณิชย์, ▁, คุมเข้ม, ร..."


## 1.2 Create JSON

In [ ]:
def createJSON(df,src_col,tgt_col):
  json_list=[]
  for i in range(len(df)):
    current = dict({"src":df.iloc[i,src_col],"tgt":df.iloc[i,tgt_col]})
    json_list.append(current)
  jsonString = json.dumps(json_list,ensure_ascii=False)
  return jsonString

In [ ]:
test_1_json = createJSON(testset,2,3)
test_2_json = createJSON(testset,4,5)

In [ ]:
print(test_1_json)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from pprint import pprint
pprint(test_1_json)

Streaming output truncated to the last 5000 lines.
 '"รายงานว่า", "หลัง", "บริษัท", "▁", "เจน", "เน", "อรัล", "▁", "มอเตอร์", '
 '"▁(", "ประเทศไทย", ")", "▁", "จํากัด", "▁", "กําหนด", "ระเบียบ", "การ", '
 '"ปฏิบัติงาน", "ของพนักงาน", "สาย", "การผลิต", "▁", "ให้", "บรรจุ", '
 '"วันเสาร์", "ซึ่งเป็น", "วันหยุด", "ของพนักงาน", "▁และ", "เป็นวันที่", '
 '"ได้", "โอ", "ที", "▁", "ให้เป็น", "วัน", "ทํางาน", "ตามปกติ", "▁แต่", '
 '"สหภาพแรงงา", "น", "ฯ", "▁ไม่", "เห็นด้วย"], ["▁และ", "ไม่สามารถ", '
 '"ตกลงกัน", "ได้", "จน", "เมื่อวัน", "ที่", "▁", "8", "▁", "กุมภาพันธ์", "▁", '
 '"2556", "▁", "บริษัทฯ", "▁", "ได้", "มีคําสั่ง", "ให้พนักงาน", "สมาชิก", '
 '"สหภาพแรงงา", "น", "หยุดงาน", "▁", "พร้อม", "ขอ", "อํานาจศาล", "ห้าม", "▁", '
 '"13", "▁", "แกนนํา", "สหภาพฯ", "▁", "เข้ามา", "ในพื้นที่", "ของ", "บริษัทฯ", '
 '"▁", "อย่าง", "เด็ดขาด"], ["▁ทําให้", "พนักงาน", "ใน", "เคร", "ื่อ", "สภาพ", '
 '"▁และ", "พนักงาน", "อื่นๆ"], ["▁", "ไม่พอใจ", "เป็น", "อย่างมาก", "▁จึงได้", '
 '"ทยอย", "กันมา", "สมั

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




 '"สาม", "ประสาน", "▁", "บีบีซี"], ["▁", "คริสเตียโน", "▁", "โรนัลโด", ",", '
 '"▁", "ก", "า", "เร็", "ธ", "▁", "เบล", "▁และ", "ค", "า", "ริม", "▁", "เบน", '
 '"เซ", "มา", "▁", "ลง", "ล่า", "ตาข่าย"], ["▁ส่วน", "โว", "ล์ฟ", "-", "ส", '
 '"บวร์ก", "จะส่ง", "แม็กซ์", "▁ครู", "เซ", "▁", "เป็นตัว", "ความหวัง", "ใน", '
 '"แดน", "หน้า", "▁", "เริ่ม", "เตะ", "▁", "01", ".45", "▁", "น", ".", "▁", '
 '"ถ่ายทอดสด", "ทาง", "ทรู", "สปอร์ต", "▁", "1", "▁และ", "ทรู", "สปอร์ต", '
 '"เอชดี"], ["▁", "2"]], "tgt": [["▁", "เรือใบสีฟ้", "า", "▁", "แมนเชสเตอร", '
 '"์", "▁", "ซิตี้", "▁", "ตัวแทน", "หนึ่งเดียว", "จาก", "อังกฤษ"], ["▁ต้อง", '
 '"เปิดบ้าน", "เอาชนะ", "▁", "เป", "แอส", "เช", "▁", "ปารีส", "▁", "แซงต์", '
 '"▁", "แชร์", "ก", "แมง", "▁", "ยอด", "ทีม", "จากเมือง", "น้ําหอม", '
 '"ให้ได้"], ["▁หาก", "ยัง", "หวัง", "เข้ารอบ", "ต่อไป"], ["▁", "ในศึก", '
 '"ยูฟ่า", "▁", "แชมเปียนส์", "ลีก", "▁", "รอบ", "▁", "8", "▁", "ทีมสุดท้าย"], '
 '["▁", "นัดที่", "▁", "2", "▁คืนนี้", "▁(", "อังคาร", "▁", "12",

## 1.3 Save JSON

In [ ]:
with open('thaisum.test.0.json', 'w', encoding='utf8') as f:
  f.write(test_1_json)

with open('thaisum.test.1.json', 'w', encoding='utf8') as f:
  f.write(test_2_json)

# files.download('thaisum.test.0.json')

# **02 Get Pytorch file from JSON**

In [ ]:
%cd /content/ThaiSum/BertSum/src

/content/ThaiSum/BertSum/src


In [ ]:
!python preprocess.py -mode format_to_bert -raw_path "/content" -save_path "/content/ThaiSum/BertSum/bert_data"  -lower -n_cpus 1 -log_file "../logs/preprocess.log"

[]
[]
[('test', '/content/thaisum.test.1.json', Namespace(dataset='', log_file='../logs/preprocess.log', lower=True, map_path='../../data/', max_src_nsents=100, max_src_ntokens_per_sent=200, max_tgt_ntokens=500, min_src_nsents=3, min_src_ntokens_per_sent=5, min_tgt_ntokens=5, mode='format_to_bert', n_cpus=1, pretrained_model='bert', raw_path='/content', save_path='/content/ThaiSum/BertSum/bert_data', select_mode='greedy', shard_size=2000, use_bert_basic_tokenizer=False), '/content/ThaiSum/BertSum/bert_data/thaisum.test.1.bert.pt'), ('test', '/content/thaisum.test.0.json', Namespace(dataset='', log_file='../logs/preprocess.log', lower=True, map_path='../../data/', max_src_nsents=100, max_src_ntokens_per_sent=200, max_tgt_ntokens=500, min_src_nsents=3, min_src_ntokens_per_sent=5, min_tgt_ntokens=5, mode='format_to_bert', n_cpus=1, pretrained_model='bert', raw_path='/content', save_path='/content/ThaiSum/BertSum/bert_data', select_mode='greedy', shard_size=2000, use_bert_basic_tokenizer=F

# **03 Test**

In [ ]:
!pip install -q bert_score
import bert_score
from bert_score import score
import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)

     |████████████████████████████████| 59 kB 4.6 MB/s 


In [ ]:
!python train.py -task ext -mode test -batch_size 3000 -test_batch_size 500 -bert_data_path "/content/ThaiSum/BertSum/bert_data/thaisum" -log_file ../logs/eval_bertsumext -model_path "/content/ThaiSum/BertSum/model_checkpoint/" -sep_optim true -use_interval true -visible_gpus 0 -max_pos 512 -max_length 200 -alpha 0.95 -min_length 50 -result_path ../results/bertsumext -test_from "/content/ThaiSum/BertSum/model_checkpoint/BertsumExt_thaisum.pt"

[2021-08-22 11:38:30,049 INFO] Loading checkpoint from /content/ThaiSum/BertSum/model_checkpoint/BertsumExt_thaisum.pt
Namespace(accum_count=1, alpha=0.95, batch_size=3000, beam_size=5, bert_data_path='/content/ThaiSum/BertSum/bert_data/thaisum', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/eval_bertsumext', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_tgt_len=140, min_length=50, mode='test', model_path='/content/ThaiSum/BertSum/model_checkpoint/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='.

## 3.1 Get BERT Score

In [ ]:
with open("/content/ThaiSum/BertSum/results/bertsumext_step0.candidate") as f: # Output Summary 
    cands = [line.strip() for line in f]

with open("/content/ThaiSum/BertSum/results/bertsumext_step0.gold") as f:  # Reference Summary
    refs = [line.strip() for line in f]

In [ ]:
P, R, F1 = score(cands, refs, lang='th', verbose=False)

print(f"System level F1 score: {F1.mean()*100:.3f}") ##  *100 to make it simpler to read similar to ROUGE.
print(f"System level P score: {P.mean()*100:.3f}")
print(f"System level R score: {R.mean()*100:.3f}")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

System level F1 score: 80.256
System level P score: 78.888
System level R score: 81.995


## 3.2 See inside

In [ ]:
with open("/content/ThaiSum/BertSum/results/bertsumext_step0.candidate") as f:
    for line in f:
        print(line)

▁ล่าสุด วันนี้ ▁ รูปพรรณ ▁ ขาย ออก บาท ละ ▁ 30 800 วันที่ ▁ 7 ▁ ส . ค . 63 ▁ ผู้สื่อข่า วรายงาน ▁ ร้านทอง บนถนน เยาวราช คึกคัก เป็นพิเศษ<q>▁เนื่องจาก มีประชาชน จํานวนมาก มา ต่อคิว ▁ นํา ทองคํา ที่เก็บ สะสม ไว้ ▁ มาขาย เพื่อ ถือ เงินสด ไว้ แทน

▁ ชนะ อ ินโดนีเซีย ▁ 3-0 ▁ เซต ▁ คว้าแชมป์ ไปครอง ส่วน ทีม หญิง ไทย<q>▁ ดีกรี อันดับ ▁ 1 ▁ ของโลก

▁ผล ช้าไป ชั่วโมง สอง ชั่วโมง ไม่เห็น เป็นไร ▁ แส ลง หู ชื่อ ▁ สมชัย ▁ ชิ่ง หนี ดื้อ ๆ ▁ พร เพชร ▁ ให้ นายกฯ ▁ ตัดสินใจ ปลดล็อก พรรค การเมือง ▁ 34 ▁ สนช . ยื่น ตีความ ต่อ วาระ ให้ผู้ ตรวจ ฯ ▁ ส่อ ขัด ▁ รธน . ▁ พท . ด า หน้า ฉะ ▁ ซือ แป๋ ▁ใคร แน่ ดูถูก ประชาชน<q>▁ แจง ไม่ได้ ล็อก เป้า แค่ คน ▁ ชินวัตร ▁ บิ๊กตู่ ▁รอ รายงาน ▁ โก ตี๋ ▁ ตาย ที่ ลาว ▁ บิ๊กป้อม ▁ โบ้ ย ศพ ใคร ไม่รู้

▁ เมิน ประเด็น จุดโทษ ปัญหา ในเกม พร ีเมียร์ลีก ▁ อังกฤษ ▁ นัดที่ แตน อาละวาด เปิดบ้าน พ่าย ▁ คริสตัล ▁ พาเลซ ▁ 0-1 ชี้ ทีม ของเขา เล่น ไม่ดีพอ เอง , สํานักข่าว ต่างประเทศ รายงาน วันที่ ▁ 27 ▁ ก . ย . ▁ ว่า ▁ กี เก ▁ ซาน เช ซ ▁ ฟลอ เรส ▁ ผู้จัดการ ทีม ชาวสเปน ▁ ของ ▁ แตน อาละว

In [ ]:
test_summary = []
with open("/content/ThaiSum/BertSum/results/bertsumext_step0.candidate") as f:
    for line in f:
      test_summary.append(line)

In [ ]:
len(test_summary)

293

In [ ]:
testset.iloc[0,1]

'มีชัย ไม่ยี่หระเสียงค้านชงสูตรพิสดารเลือกตั้ง ส.ส. ประกาศ ผลช้าไปชั่วโมงสองชั่วโมงไม่เห็นเป็นไร แสลงหูชื่อ สมชัย ชิ่งหนีดื้อๆ พรเพชร ให้นายกฯ ตัดสินใจปลดล็อกพรรคการเมือง'